In [1]:
!pip install 'accelerate>=0.26.0' torch transformers


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np

class Agent:
    def __init__(self, name, role, model_name):
        self.name = name
        self.role = role
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=100,  # Limit the response length
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        prompt += "\nAssistant:"
        response = self.pipeline(prompt, max_new_tokens=100)[0]['generated_text']
        response = response[len(prompt):].strip().split('Human:')[0].strip()
        return response

    def update_score(self, correct):
        if correct:
            self.score += 1.0
        else:
            self.score = max(0.1, self.score - 0.5)  # Prevent negative weights

def construct_message(agents, idx, question, round_num, final=False, self_reflection=False):
    messages = []
    current_agent = agents[idx]
    
    # Add system message
    messages.append({
        'role': 'system',
        'content': f"You are Agent {current_agent.name}, an AI assistant specialized in {current_agent.role}. "
                   f"Participate in a collaborative problem-solving discussion to answer the given question."
    })
    
    # Add question
    messages.append({'role': 'user', 'content': f"Question: {question}"})
    
    # Add previous rounds of discussion
    for i, entry in enumerate(current_agent.history):
        messages.append({'role': 'assistant', 'content': f"Round {i + 1}: {entry['response']}"})
    
    # Construct the prompt based on the current situation
    if self_reflection:
        prompt = (
            f"This is round {round_num}. Reflect on your previous answer and the responses of other agents. "
            f"Can you improve or correct your previous answer? If so, how? If not, why not?"
        )
    elif final:
        prompt = (
            f"Based on all discussions, what is your final answer to the question: {question}? "
            f"Provide detailed reasoning."
        )
    else:
        prompt = (
            f"This is round {round_num}. Consider the previous discussions and provide your perspective "
            f"on the question: {question}. Focus on your specialization in {current_agent.role}."
        )
    
    messages.append({'role': 'user', 'content': prompt})
    
    return messages

def run_debate(agents, question, max_rounds=3):
    for round_num in range(1, max_rounds + 1):
        print(f"\n=== Round {round_num} ===")
        for agent in agents:
            messages = construct_message(agents, agent.name, question, round_num)
            response = agent.generate_response(messages)
            agent.history.append({'round': round_num, 'response': response})
            print(f"{agent.name} ({agent.role}): {response}\n")
        
        # Self-reflection
        for agent in agents:
            messages = construct_message(agents, agent.name, question, round_num, self_reflection=True)
            reflection = agent.generate_response(messages)
            print(f"{agent.name} reflection: {reflection}\n")
        
    # Final answers
    print("\n=== Final Answers ===")
    for agent in agents:
        messages = construct_message(agents, agent.name, question, max_rounds, final=True)
        final_answer = agent.generate_response(messages)
        print(f"{agent.name} final answer: {final_answer}\n")

def calculate_consensus(responses):
    # Simple consensus calculation based on response similarity
    processed_responses = [response.strip().lower() for response in responses]
    unique_responses = set(processed_responses)
    consensus_score = 1.0 / len(unique_responses)
    return consensus_score

In [3]:
from huggingface_hub import login
import os

# Login
login()

In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np

class Agent:
    def __init__(self, name, role, model_name):
        self.name = name
        self.role = role
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map='auto',
            torch_dtype=torch.float16,
            low_cpu_mem_usage=True
        )
        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            max_new_tokens=1024,  # Limit the response length
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        prompt = "\n".join([f"{m['role']}: {m['content']}" for m in messages])
        prompt += "\nAssistant:"
        response = self.pipeline(prompt, max_new_tokens=100)[0]['generated_text']
        response = response[len(prompt):].strip().split('Human:')[0].strip()
        return response

    def update_score(self, correct):
        if correct:
            self.score += 1.0
        else:
            self.score = max(0.1, self.score - 0.5)  # Prevent negative weights

def construct_message(agent, question, round_num, final=False, self_reflection=False):
    messages = []
    
    # Add system message
    messages.append({
        'role': 'system',
        'content': f"You are Agent {agent.name}, an AI assistant specialized in {agent.role}. "
                   f"Participate in a collaborative problem-solving discussion to answer the given question."
    })
    
    # Add question
    messages.append({'role': 'user', 'content': f"Question: {question}"})
    
    # Add previous rounds of discussion
    for i, entry in enumerate(agent.history):
        messages.append({'role': 'assistant', 'content': f"Round {i + 1}: {entry['response']}"})
    
    # Construct the prompt based on the current situation
    if self_reflection:
        prompt = (
            f"This is round {round_num}. Reflect on your previous answer and the responses of other agents. "
            f"Can you improve or correct your previous answer? If so, how? If not, why not?"
        )
    elif final:
        prompt = (
            f"Based on all discussions, what is your final answer to the question: {question}? "
            f"Provide detailed reasoning."
        )
    else:
        prompt = (
            f"This is round {round_num}. Consider the previous discussions and provide your perspective "
            f"on the question: {question}. Focus on your specialization in {agent.role}."
        )
    
    messages.append({'role': 'user', 'content': prompt})
    
    return messages

def run_debate(agents, question, max_rounds=3):
    for round_num in range(1, max_rounds + 1):
        print(f"\n=== Round {round_num} ===")
        for agent in agents:
            messages = construct_message(agent, question, round_num)
            response = agent.generate_response(messages)
            agent.history.append({'round': round_num, 'response': response})
            print(f"{agent.name} ({agent.role}): {response}\n")
        
        # Self-reflection
        for agent in agents:
            messages = construct_message(agent, question, round_num, self_reflection=True)
            reflection = agent.generate_response(messages)
            print(f"{agent.name} reflection: {reflection}\n")
        
    # Final answers
    print("\n=== Final Answers ===")
    for agent in agents:
        messages = construct_message(agent, question, max_rounds, final=True)
        final_answer = agent.generate_response(messages)
        print(f"{agent.name} final answer: {final_answer}\n")

def calculate_consensus(responses):
    # Simple consensus calculation based on response similarity
    processed_responses = [response.strip().lower() for response in responses]
    unique_responses = set(processed_responses)
    consensus_score = 1.0 / len(unique_responses)
    return consensus_score

# Usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'
agents = [
    Agent(
        name='Agent1',
        role='mathematical reasoning',
        model_name='meta-llama/Llama-3.2-1b-Instruct'
    ),
    Agent(
        name='Agent2',
        role='logical consistency',
        model_name='Qwen/Qwen2.5-0.5B'
    ),
    # Agent(
    #     name='Agent3',
    #     role='context integration',
    #     model_name='deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct'
    # ),
]
question = "What is the sum of all even numbers between 1 and 100?"
run_debate(agents, question)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

...

Agent2 reflection: I apologize, but as an AI assistant, I cannot provide a specific answer to the question as it is not clear what the question is asking for. However, if you would like me to help with the previous sum of all even numbers between 1 and 100, I would be happy to provide an explanation.
As per the previous discussion, the sum of all even numbers between 1 and 100 is 5050. However, the question asks for the sum


=== Round 3 ===
Agent1 (mathematical reasoning): Round 4: Thank you for the reminder. I'll provide my perspective on the problem.

To calculate the sum of all even numbers between 1 and 100, I'll focus on the following:

First, I'll find the smallest and largest even numbers in the given range: 2 and 100.
Next, I'll calculate the number of even numbers between these two values: (100 - 2) / 2 + 1 = 50.
Now, I'll use the



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Agent2 (logical consistency): Round 3: The sum of all even numbers between 1 and 100 is 5050. Based on the previous discussions, the sum of all even numbers between 1 and 100 is 5050. However, the question asks for the sum of all even numbers between 1 and 100, which is not the same as the previous sum. The sum of all even numbers between 1 and 100 is calculated by

Agent1 reflection: Round 5: I'd like to reflect on my previous answers. Upon reviewing the previous responses, I noticed that I provided two different calculations for the sum of even numbers between 1 and 100.

The correct calculation should be:
Sum = (50/2) * (2 + 100) = 25 * 102 = 2550

The mistake was in the calculation of the number of even numbers. I should have used the formula (100 - 2) / 2



Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Agent2 reflection: I understand the question is asking for the sum of all even numbers between 1 and 100. I would like to provide a more detailed answer to help clarify the problem. 
To find the sum of all even numbers between 1 and 100, we can use the formula for the sum of an arithmetic series. The formula is: 
Sum = (n/2)(a + l)
where n is the number of terms, a is the first term, and l


=== Final Answers ===
Agent1 final answer: Round 5: Based on all discussions, my final answer to the question is as follows:

The sum of all even numbers between 1 and 100 can be calculated using the following formula:

Sum = (Sum of first term) × (Number of terms) / 2

In this case, the first term is 2, the last term is 100, and the number of terms is 50 (calculated as (100 - 2) / 2 + 1).

Agent2 final answer: Based on all discussions, the sum of all even numbers between 1 and 100 is 5050.



In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import numpy as np
import logging

logging.basicConfig(level=logging.INFO)

class Agent:
    def __init__(self, name, role, model_name):
        self.name = name
        self.role = role
        self.model_name = model_name
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Load tokenizer and model
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.float16 if self.device.type == 'cuda' else torch.float32
            ).to(self.device)
        except Exception as e:
            logging.error(f"Failed to load model {self.model_name}: {e}")
            raise

        self.pipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=self.device,
            max_new_tokens=500,
            do_sample=False,
            temperature=0.7,
            top_p=0.9,
        )
        self.history = []
        self.score = 1.0

    def generate_response(self, messages):
        prompt = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in messages])
        prompt += "\nAssistant:"
        response = self.pipeline(prompt)[0]['generated_text']
        # Extract the assistant's response
        response = response[len(prompt):].strip().split('\n')[0].strip()
        return response

    def update_score(self, correct):
        if correct:
            self.score += 1.0
        else:
            self.score = max(0.1, self.score - 0.5)

def construct_message(agent, question, round_num, final=False, self_reflection=False):
    messages = []

    # Add system message
    messages.append({
        'role': 'system',
        'content': f"You are {agent.name}, an AI assistant specialized in {agent.role}. "
                   f"Participate in a collaborative problem-solving discussion to answer the given question."
    })

    # Add question
    messages.append({'role': 'user', 'content': f"Question: {question}"})

    # Add previous rounds of discussion
    for i, entry in enumerate(agent.history):
        messages.append({'role': 'assistant', 'content': f"Round {entry['round']}: {entry['response']}"})

    # Construct the prompt based on the current situation
    if self_reflection:
        prompt = (
            f"This is round {round_num}. Reflect on your previous answer and the responses of other agents. "
            f"Can you improve or correct your previous answer? If so, how? If not, why not?"
        )
    elif final:
        prompt = (
            f"Based on all discussions, what is your final answer to the question: {question}? "
            f"Provide detailed reasoning."
        )
    else:
        prompt = (
            f"This is round {round_num}. Consider the previous discussions and provide your perspective "
            f"on the question: {question}. Focus on your specialization in {agent.role}."
        )

    messages.append({'role': 'user', 'content': prompt})

    return messages

def run_debate(agents, question, max_rounds=3):
    for round_num in range(1, max_rounds + 1):
        print(f"\n=== Round {round_num} ===")
        for agent in agents:
            messages = construct_message(agent, question, round_num)
            response = agent.generate_response(messages)
            agent.history.append({'round': round_num, 'response': response})
            print(f"{agent.name} ({agent.role}): {response}\n")

        # Self-reflection
        for agent in agents:
            messages = construct_message(agent, question, round_num, self_reflection=True)
            reflection = agent.generate_response(messages)
            print(f"{agent.name} reflection: {reflection}\n")

    # Final answers
    print("\n=== Final Answers ===")
    final_responses = []
    for agent in agents:
        messages = construct_message(agent, question, max_rounds, final=True)
        final_answer = agent.generate_response(messages)
        final_responses.append(final_answer)
        print(f"{agent.name} final answer: {final_answer}\n")

    # Calculate consensus
    consensus_score = calculate_consensus(final_responses)
    print(f"Consensus Score: {consensus_score}")

def calculate_consensus(responses):
    """
    Calculates a simple consensus score based on the uniqueness of responses.
    """
    processed_responses = [response.strip().lower() for response in responses]
    unique_responses = set(processed_responses)
    consensus_score = 1.0 / len(unique_responses)
    return consensus_score

if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    agents = [
        Agent(
        name='Agent1',
        role='mathematical reasoning',
        model_name='meta-llama/Llama-3.1-8B-Instruct'
        ),
        Agent(
            name='Agent2',
            role='logical consistency',
            model_name='Qwen/Qwen2.5-Coder-7B-Instruct'
        ),
    ]
    question = "What is the sum of all even numbers between 1 and 100?"
    run_debate(agents, question)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

ERROR:root:Failed to load model meta-llama/Llama-3.1-8B-Instruct: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacty of 44.34 GiB of which 446.81 MiB is free. Process 2371953 has 43.89 GiB memory in use. Of the allocated memory 43.18 GiB is allocated by PyTorch, and 406.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF


OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacty of 44.34 GiB of which 446.81 MiB is free. Process 2371953 has 43.89 GiB memory in use. Of the allocated memory 43.18 GiB is allocated by PyTorch, and 406.24 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF